<a href="https://colab.research.google.com/github/krishnakaushik25/handwritten_digit_recognizer-DL/blob/master/Transformer%2BSVM%2BSemantically_similar_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pipeline of the model                  
Text cleaning/preprocessing(+) + Transformer + Support Vector Machine + Majority voting for semantically similar but labelled wrong tweets + Filtering basing on keywords(words directly implying to disaster)

In [ ]:
!pip install tensorflow-text==2.0.0 --user

     |████████████████████████████████| 9.1MB 9.6MB/s 
     |████████████████████████████████| 86.4MB 75kB/s 
     |████████████████████████████████| 450kB 49.8MB/s 
     |████████████████████████████████| 3.8MB 45.7MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=000f91b2290b48b6c827c3cd9b6b506b459b6296698d2f1dc1648eb746b5ab89
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
import time
start_time = time.time()

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from tqdm import tqdm
import numpy as np
import pandas as pd
import re

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub


In [ ]:
#print full tweet , not a part
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 310)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


Loading data

In [ ]:
train = pd.read_csv("/content/drive/My Drive/train.csv")
test = pd.read_csv("/content/drive/My Drive/test.csv")
submission = pd.read_csv("/content/drive/My Drive/sample_submission (2).csv")
train_copy = train.copy()
length_train = len(train.index)
length_train

7613

mislabelled examples


All the records are from the original train set before a text cleaning. The examples motivate the regex for the 'text' cleaning/preprocessing.

In [ ]:
index_hot = [4415, 4400, 4399,4403,4397,4396, 4394,4414, 4393,4392,4404,4407,4420,4412,4408,4391,4405]
train_example = train.copy()
train_example.loc[index_hot,:]

,id,keyword,location,text,target
4415,6276,hijacking,Japan,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/xV3D9bPjHi #prebreak #best,1
4400,6256,hijacking,Brazil,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/aAtt5aMnmD #prebreak #best,0
4399,6255,hijacking,EastCarolina,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/nQiObcZKrT #prebreak #best,0
4403,6259,hijacking,NaN,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/qj3PVgaVN7 #prebreak #best,1
4397,6253,hijacking,rome,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/J5onxFwLAo #prebreak #best,0
4396,6248,hijacking,"Chiyoda Ward, Tokyo",#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/wvTPuRYx63 #prebreak #best,0
4394,6246,hijacking,China,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/cx6auPneMu #prebreak #best,0
4414,6274,hijacking,NaN,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/cOMuiOk3mP #prebreak #best,1
4393,6245,hijacking,"brisbane, australia",#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/s4PNIhJQX7 #prebreak #best,0
4392,6244,hijacking,Mongolia,#hot Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/J2aQs5loxu #prebreak #best,1


In [ ]:
index_hot = [6840,6834,6837,6841,6816]
train_example = train.copy()
train_example.loc[index_hot,:]

,id,keyword,location,text,target
6840,9797,trapped,NaN,Hollywood Movie About Trapped Miners Released in Chile: 'The 33' Hollywood movie about trapped miners starring... http://t.co/0f8XA4Ih1U,1
6834,9788,trapped,å_å_Los Mina Cityã¢,Hollywood Movie About Trapped Miners Released in Chile: 'The 33' Hollywood movie about trapped miners starring... http://t.co/x8moYeVjsJ,0
6837,9794,trapped,NaN,Hollywood Movie About Trapped Miners Released in Chile: 'The 33' Hollywood movie about trapped miners starring... http://t.co/tyyfG4qQvM,1
6841,9800,trapped,Like us on Face,Hollywood Movie About Trapped Miners Released in Chile: 'The 33' Hollywood movie about trapped miners starring... http://t.co/3Yu26V19zh,0
6816,9763,trapped,Orlando,Hollywood Movie About Trapped Miners Released in Chile: 'The 33' Hollywood movie about trapped miners starring... http://t.co/KK8cnppZMk,0


In [ ]:
index_hot = [6828,6831]
train_example = train.copy()
train_example.loc[index_hot,:]

,id,keyword,location,text,target
6828,9779,trapped,worldwide,Hollywood Movie About Trapped Miners Released in Chile http://t.co/EXQKmlg4NJ,0
6831,9783,trapped,Puerto Rico,Hollywood Movie About Trapped Miners Released in Chile http://t.co/qkrLtrd39B,1


In [ ]:
index_hot = [591, 587]
train_example = train.copy()
train_example.loc[index_hot,:]

,id,keyword,location,text,target
591,853,bioterror,NaN,#world FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/5zDbTktwW7,0
587,848,bioterror,iTunes,#world FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/wvExJjRG6E,1


In [ ]:
index_hot = [601,576,584,608,606,603,592,604]
train_example = train.copy()
train_example.loc[index_hot,:]

,id,keyword,location,text,target
601,870,bioterror,"Melbourne, Florida",FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/c0p3SEsqWm via @usatoday,1
576,834,bioterror,"West Virginia, USA",FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/HQsU8LWltH via @usatoday,1
584,843,bioterror,"Wilmington, DE",FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/qZQc8WWwcN via @usatoday,0
608,877,bioterror,NaN,FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/4M5UHeyfDo via @USATODAY,1
606,875,bioterror,NaN,FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/hrqCJdovJZ,0
603,872,bioterror,NaN,FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/MqbYrAvK6h,1
592,856,bioterror,US,FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/pWAMG8oZj4,1
604,873,bioterror,Over the Moon...,#FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/S4SiCMYRmH,1


In [ ]:
index_hot = [3667,3674,3688,3696]
train_example = train.copy()
train_example.loc[index_hot,:]

,id,keyword,location,text,target
3667,5221,fatality,Nairobi,Fatality!,0
3674,5229,fatality,"Enterprise, NV",Fatality https://t.co/407V1y4HHg,0
3688,5248,fatality,NaN,Fatality https://t.co/GF5qjGoyCi,1
3696,5259,fatality,Honduras,Fatality ????,0


In [ ]:
index_hot = [3913,3914,3936,3921,3941,3937,3938]
train_example = train.copy()
train_example.loc[index_hot,:]

,id,keyword,location,text,target
3913,5564,flood,New York,Spot Flood Combo 53inch 300W Curved Cree LED Work Light Bar 4X4 Offroad Fog Lamp - Full reÛ_ http://t.co/xxkHjySn0p http://t.co/JEVHKNJGBX,1
3914,5565,flood,New York,Spot Flood Combo 53inch 300W Curved Cree LED Work Light Bar 4X4 Offroad Fog Lamp - Full reÛ_ http://t.co/jCDd6SD6Qn http://t.co/9gUCkjghms,0
3936,5595,flood,New York,Spot Flood Combo 53inch 300W Curved Cree LED Work Light Bar 4X4 Offroad Fog Lamp - Full reÛ_ http://t.co/5xmCE6JufS http://t.co/3Zo7PX3p1V,0
3921,5577,flood,New York,Spot Flood Combo 53inch 300W Curved Cree LED Work Light Bar 4X4 Offroad Fog Lamp - Full reÛ_ http://t.co/mTmoIa0Oo0 http://t.co/Nn4ZtCmSRU,0
3941,5603,flood,New York,Spot Flood Combo 53inch 300W Curved Cree LED Work Light Bar 4X4 Offroad Fog Lamp - Full reÛ_ http://t.co/6zCfHi7Srw http://t.co/vWYkDaU1vm,0
3937,5598,flood,New York,Spot Flood Combo 53inch 300W Curved Cree LED Work Light Bar 4X4 Offroad Fog Lamp - Full reÛ_ http://t.co/O097vSOtxk http://t.co/I23Xy7iEjj,0
3938,5599,flood,New York,Spot Flood Combo 53inch 300W Curved Cree LED Work Light Bar 4X4 Offroad Fog Lamp - Full reÛ_ http://t.co/fDSaoOiskJ http://t.co/2uVmq4vAfQ,0


In [ ]:
index_hot = [3136, 3133]
train_example = train.copy()
train_example.loc[index_hot,:]

,id,keyword,location,text,target
3136,4507,emergency,New York,Survival Kit Whistle Fire Starter Wire Saw Cree Torch Emergency Blanket S knife - Full reÛ_ http://t.co/2OroYUNYM2 http://t.co/C9JnXz3DXC,0
3133,4499,emergency,New York,Survival Kit Whistle Fire Starter Wire Saw Cree Torch Emergency Blanket S knife - Full reÛ_ http://t.co/cm7HqwWUlZ http://t.co/KdwAzHQTov,1


In [ ]:
index_hot = [3930,3933,3924,3917]
train_example = train.copy()
train_example.loc[index_hot,:]

,id,keyword,location,text,target
3930,5588,flood,New York,2pcs 18W CREE Led Work Light Offroad Lamp Car Truck Boat Mining 4WD FLOOD BEAM - Full reaÛ_ http://t.co/O1SMUh2unn http://t.co/xqj6WgiuQH,0
3933,5592,flood,New York,2pcs 18W CREE Led Work Light Offroad Lamp Car Truck Boat Mining 4WD FLOOD BEAM - Full reaÛ_ http://t.co/1QT51r5h98 http://t.co/OQH1JbUEnl,0
3924,5580,flood,New York,2pcs 18W CREE Led Work Light Offroad Lamp Car Truck Boat Mining 4WD FLOOD BEAM - Full reaÛ_ http://t.co/ApWXS5Mm44 http://t.co/DS76loZLSu,1
3917,5570,flood,NaN,2pcs 18W CREE Led Work Light Offroad Lamp Car Truck Boat Mining 4WD FLOOD BEAM - Full reaÛ_ http://t.co/VDeFmulx43 http://t.co/yqpAIjSa5g,0


In [ ]:
index_hot = [246,270,266,259,253,251,250,271]
train_example = train.copy()
train_example.loc[index_hot,:]

,id,keyword,location,text,target
246,349,annihilation,NaN,U.S National Park Services Tonto National Forest: Stop the Annihilation of the Salt River Wild Horse... http://t.co/6LoJOoROuk via @Change,0
270,393,annihilation,BIG D HOUSTON/BOSTON/DENVER,U.S National Park Services Tonto National Forest: Stop the Annihilation of the Salt River Wild Horse... https://t.co/0fekgyBY5F via @Change,0
266,384,annihilation,"Tacoma,Washington",U.S National Park Services Tonto National Forest: Stop the Annihilation of the Salt River Wild Horse... https://t.co/x2Wn7O2a3w via @Change,0
259,370,annihilation,"Coolidge, AZ",U.S National Park Services Tonto National Forest: Stop the Annihilation of the Salt River Wild Horse... https://t.co/MatIJwkzbh via @Change,0
253,360,annihilation,NaN,U.S National Park Services Tonto National Forest: Stop the Annihilation of the Salt River Wild Horse... http://t.co/KPQk0C4G0M via @Change,1
251,356,annihilation,CA physically- Boston Strong?,U.S National Park Services Tonto National Forest: Stop the Annihilation of the Salt River Wild Horse... https://t.co/sW1sBua3mN via @Change,1
250,355,annihilation,NaN,U.S National Park Services Tonto National Forest: Stop the Annihilation of the Salt River Wild Horse... https://t.co/m8MvDSPJp7 via @Change,0
271,394,annihilation,"Chandler, AZ",U.S National Park Services Tonto National Forest: Stop the Annihilation of the Salt River Wild Horse... http://t.co/SB5R7ShcCJ via @Change,1


Data cleaning/preprosessing

In [ ]:
def new_line(text):
    text = re.sub(r'\t', ' ', text) # remove tabs
    text = re.sub(r'\n', ' ', text) # remove line jump
    return text

URL can not be truncated in a tweet, and if a string is too long, it is truncated at position just before the URL and ... is added. if such contracted 'words' have 3 or less letters, the words are considered as noise and are eliminated.

In [ ]:
def url(text):
# quite many tweets are truncated like "Experts in France 
# begin examining airplane debris found on Reunion Island: French air 
# accident experts o... http://t.co/YVVPznZmXg #news" , the explanation is above
    text = re.sub(r' \w{1,3}\.{3,3} http\S{0,}', ' ', text)
    text = re.sub(r' \w{1,3}Û_ http\S{0,}', ' ', text)
# some symbols and words one space before 'http' are eliminated, it is assumed the words have no a 
# semantical meaning and predictive power in the position. 
    text = re.sub(r"mp3 http\S{0,}", r" ", text)
    text = re.sub(r"rar http\S{0,}", r" ", text)
    pattern = re.compile(r'( pin\:\d+ | via )http\S{0,}')
    text = pattern.sub(r' ', text)
# the pattern in tweet context have no a big meaning and the elimination of the words 
# unify the strings structure 
    pattern = re.compile(r'Full read by|Full read b|Full read|Full rea|Full re|Full r')
    text = pattern.sub(r' ', text)
    pattern = re.compile(r'Full story at|Full story a|Full story|Full stor|Full sto|Full st|Full s')
    text = pattern.sub(r' ', text)
    
    return text


In [ ]:
def clean(text):    
    text = new_line(text)
# eliminate the pattern
    text = re.sub(r'(&amp;|&gt;|&lt;)', " ", text)
    text = re.sub(r"\s+", " ", text) # remove extra spaces
    text = url(text)
    
# the pattern is 'translated as 'USER'
# in https://www.kaggle.com/quentinsarrazin/tweets-preprocessing similar 'translation' is used
# in https://arxiv.org/ftp/arxiv/papers/1807/1807.07752.pdf similar pattern 
# is 'translated as 'USER_NAME'
    text = re.sub(r'@\S{0,}', ' USER ', text)
    text = re.sub(r"\s+", " ", text) # remove extra spaces  
# shrink multiple USER USER USER ... to USER
    text = re.sub(r'\b(USER)( \1\b)+', r'\1', text)
    
# multiple  letters repeats like in 'Oooooohhh' are truncated to 2 letters, not possible to truncate 
# to 1 letter, because it may generated false meaning like  'good' to 'god'
    text = re.sub(r'([a-zA-Z])\1{1,}', r'\1\1', text)
    
#  URLs , if not yet eliminated by url function are eliminated 
    text = re.sub(r"htt\S{0,}", " ", text)
    
# remove all characters if not in the list [a-zA-Z\d\s]
    text = re.sub(r"[^a-zA-Z\d\s]", " ", text)
    
# the digit(s) pattern is 'translated' to 'NUMBER'
# in https://www.kaggle.com/quentinsarrazin/tweets-preprocessing similar 'translation' is used
    text = re.sub(r'^\d\S{0,}| \d\S{0,}| \d\S{0,}$', ' NUMBER ', text)
    text = re.sub(r"\s+", " ", text) # remove extra spaces 
# shrink multiple NUMBER NUMBER  ... to NUMBER
    text = re.sub(r'\b(NUMBER)( \1\b)+', r'\1', text)
    
# remove digits if not eliminated above in 'NUMBER translation'
    text = re.sub(r"[0-9]", " ", text)
    
    text = text.strip() # remove spaces at the beginning and at the end of string    
# to reveal more equivalence classes the ' via USER' at the end of string is eliminated
    text = re.sub(r' via\s{1,}USER$', ' ', text)
    
    text = re.sub(r"\s+", " ", text) # remove extra spaces
    text = text.strip() # remove spaces at the beginning and at the end of string
    
    return text

In [ ]:
train['text'][5450:5550] #before cleaning display of  train text 

5450    Police kill gunman at Nashville movie theatre - Times of India: Times of India Police killÛ_ http://t.co/K8BExkgwr2                          
5451    Oops: Bounty hunters try to raid Phoenix police chief's home http://t.co/u30n3fFX8Y                                                           
5452    @robbiewilliams U fkn asswipe playing for Israeli child killers.. The fkn karma police will get U.                                            
5453    New technology designed to help prevent dangerous police chases: In an effort to reduce injuries Milwaukee policeÛ_ http://t.co/cedjdlPDAN   
5454    @ArizonaDOT Price Rd North bound closed from University to Rio Salado.. Lots of police..  What's crackin?                                     
5455    Selection September Bridgeport men charged in home invasion Police looking for burglar. - http://t.co/7mlCD0l0b8                              
5456    Police walk up on me I be blowin smoke in dey face  wanna lock me up cus I got dope sh

In [ ]:
train.text = train.text.apply(clean)
test.text = test.text.apply(clean)

Cut lengthy ends of longest tweets

In [ ]:
max_length_tr = train.text.map(len).max()
max_length_te = test.text.map(len).max()
max_length = max(max_length_tr, max_length_te)

print("At the stage of text processing:")
print(f"...the size of longest text string in train set is  {max_length_tr}")
print(f"...the size of longest text string in test set is  {max_length_te}")

At the stage of text processing:
...the size of longest text string in train set is  162
...the size of longest text string in test set is  151


In [ ]:
# the new max possible length will be (max_length - delta) , strings longer than new_max will be 
# decreased to new_max 
def cut(max_len, delta, x):
    new_max = max_len - delta
    length = len(x)
    if length <= new_max:
        return x 
    else:
        return x[:(new_max-length)]
    

delta = 25 
train.text = train.text.map(lambda x: cut(max_length, delta, x))
test.text = test.text.map(lambda x: cut(max_length, delta, x))

new_max_length_tr = train.text.map(len).max()
new_max_length_te = test.text.map(len).max()

print("After we cut tails of the longest tweets:")
print(f"...the size of longest text string in train set is  {new_max_length_tr}")
print(f"...the size of longest text string in test set is  {new_max_length_te}")

After we cut tails of the longest tweets:
...the size of longest text string in train set is  137
...the size of longest text string in test set is  137


In [ ]:
train['text'][5450:5550] # after cleaning dispalay the 'text' 

5450    Police kill gunman at Nashville movie theatre Times of India Times of India Police kill                                                  
5451    Oops Bounty hunters try to raid Phoenix police chief s home                                                                              
5452    USER U fkn asswipe playing for Israeli child killers The fkn karma police will get U                                                     
5453    New technology designed to help prevent dangerous police chases In an effort to reduce injuries Milwaukee police                         
5454    USER Price Rd North bound closed from University to Rio Salado Lots of police What s crackin                                             
5455    Selection September Bridgeport men charged in home invasion Police looking for burglar                                                   
5456    Police walk up on me I be blowin smoke in dey face wanna lock me up cus I got dope shit is gay                      

Equivalence classes with mislabelling

In [ ]:
df_mislabeled = train.groupby(['text']).nunique().sort_values(by='target', ascending=False)
df_mislabeled = df_mislabeled[df_mislabeled['target'] > 1]['target']
index_misl = df_mislabeled.index.tolist()

lenght = len(index_misl)

print(f"There are {lenght} equivalence classes with mislabelling")

There are 85 equivalence classes with mislabelling


In [ ]:
index_misl

['Why are you deluged with low self image Take the quiz',
 'That horrible sinking feeling when you ve been at home on your phone for a while and you realise its been on NUMBER this whole time',
 'Survival Kit Whistle Fire Starter Wire Saw Cree Torch Emergency Blanket S knife Full',
 'World War II book LIGHTNING JOE An Autobiography by General J Lawton Collins',
 'Caution breathing may be hazardous to your health',
 'foodscare offers go NestleIndia slips into loss after Magginoodle ban unsafe and hazardous for humanconsumption',
 'USER FETTILOOTCH IS SLANGLUCCI OPPRESSIONS GREATEST DANGER COMING SOON THE ALBUM',
 'World Annihilation vs Self Transformation Aliens Attack to Exterminate Humans',
 'The way you move is like a full on rainstorm and I m a house of cards',
 'To fight bioterrorism sir',
 'How is it one careless match can start a forest fire but it takes a whole box to start a campfire',
 'Deaths NUMBER',
 'Sassy city girl country hunk stranded in Smoky Mountain snowstorm AoMS ib

In [ ]:
train_nu_target = train[train['text'].isin(index_misl)].sort_values(by = 'text')
#train_nu_target.head(60)
train_nu_target[0:309]

,id,keyword,location,text,target
7091,10156,upheaval,Connecticut,A look at state actions a year after Ferguson s upheaval,1
7068,10124,upheaval,Oregon,A look at state actions a year after Ferguson s upheaval,0
7059,10113,upheaval,NaN,A look at state actions a year after Ferguson s upheaval,0
7056,10110,upheaval,Oregon,A look at state actions a year after Ferguson s upheaval,0
4312,6123,hellfire,?????? ???? ??????,Allah describes piling up wealth thinking it would last forever as the description of the people of Hellfire in Surah Humaza Reflect,1
4299,6105,hellfire,?????? ??? ?????? ????????,Allah describes piling up wealth thinking it would last forever as the description of the people of Hellfire in Surah Humaza Reflect,0
4290,6094,hellfire,"Jubail IC, Saudi Arabia.",Allah describes piling up wealth thinking it would last forever as the description of the people of Hellfire in Surah Humaza Reflect,0
2823,4058,displaced,"Ojodu,Lagos",Angry Woman Openly Accuses NEMA Of Stealing Relief Materials Meant For IDPs An angry Internally Displaced,0
2836,4080,displaced,Nigeria,Angry Woman Openly Accuses NEMA Of Stealing Relief Materials Meant For IDPs An angry Internally Displaced,1
2846,4092,displaced,Nigeria,Angry Woman Openly Accuses NEMA Of Stealing Relief Materials Meant For IDPs An angry Internally Displaced,0


In [ ]:
num_records = train_nu_target.shape[0]
length = len(index_misl)
print(f"There are {num_records} records in train set which are split in {lenght} equivalence classes (with mislabelling)") 

There are 309 records in train set which are split in 85 equivalence classes (with mislabelling)


In [ ]:
copy = train_nu_target.copy()
classes = copy.groupby('text').agg({'keyword':np.size, 'target':np.mean}).rename(columns={'keyword':'Number of records in train set', 'target':'Target mean'})

classes.sort_values('Number of records in train set', ascending=False).head(100)

,Number of records in train set,Target mean
text,,
hot Funtenna hijacking computers to send data as sound waves Black Hat NUMBER prebreak best,17,0.294118
Pandemonium In Aba As Woman Delivers Baby Without Face Photos,10,0.400000
Reddit Will Now Quarantine Offensive Content,10,0.300000
U S National Park Services Tonto National Forest Stop the Annihilation of the Salt River Wild Horse,8,0.375000
FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps,8,0.750000
hot C NUMBER specially modified to land in a stadium and rescue hostages in Iran in NUMBER prebreak best,8,0.875000
Remove the and Linkury Browser Hijacker,7,0.285714
Truth News BBC CNN Islam Truth god ISIS terrorism Quran Lies,7,0.571429
Spot Flood Combo NUMBER Curved Cree LED Work Light Bar NUMBER Offroad Fog Lamp Full,7,0.142857


Majority voting:
If target mean is lower than 0.5 , I relabel to 0, otherwise to 1

In [ ]:
majority_df = train_nu_target.groupby(['text'])['target'].mean()
len(majority_df.index)

85

In [ ]:
majority_df = train_nu_target.groupby(['text'])['target'].mean()
len(majority_df.index)

85

In [ ]:
def relabel(r, majority_index):
    ind = ''
    if r['text'] in majority_index:
        ind = r['text']
#        print(ind)
        if majority_df[ind] < 0.5:
            return 0
        else:
            return 1
    else: 
        return r['target']

In [ ]:
train['target'] = train.apply( lambda row: relabel(row, majority_df.index), axis = 1)

In [ ]:
new_df = train[train['text'].isin(majority_df.index)].sort_values(['target', 'text'], ascending = [False, True])
new_df.head(310)

,id,keyword,location,text,target
1164,1676,bridge%20collapse,"New Delhi, Delhi",Ashes NUMBER Australia s collapse at Trent Bridge among worst in history England bundled out Australia for NUMBER,1
1186,1707,bridge%20collapse,NaN,Ashes NUMBER Australia s collapse at Trent Bridge among worst in history England bundled out Australia for NUMBER,1
4597,6537,injury,NaN,CLEARED incident with injury I NUMBER inner loop Exit NUMBER MD NUMBER Georgia Ave Silver Spring,1
4605,6548,injury,NaN,CLEARED incident with injury I NUMBER inner loop Exit NUMBER MD NUMBER Georgia Ave Silver Spring,1
4618,6566,injury,NaN,CLEARED incident with injury I NUMBER inner loop Exit NUMBER MD NUMBER Georgia Ave Silver Spring,1
4232,6012,hazardous,NaN,Caution breathing may be hazardous to your health,1
4235,6017,hazardous,NaN,Caution breathing may be hazardous to your health,1
4182,5939,hazard,"Alameda, CA",Choking Hazard Prompts Recall Of Kraft Cheese Singles,1
4196,5959,hazard,"Pleasanton, CA",Choking Hazard Prompts Recall Of Kraft Cheese Singles,1
5785,8254,rioting,"Vista, CA",Cindy Noonan USER in Baltimore Rioting YAHIstorical UndergroundRailraod,1


In [ ]:
df_mislabeled = train.groupby(['text']).nunique().sort_values(by='target', ascending=False)
df_mislabeled = df_mislabeled[df_mislabeled['target'] > 1]['target']
index_misl = df_mislabeled.index.tolist()
#index_dupl[0:50]
len(index_misl)

0

universal 
Multilingual Encoder module

In [ ]:
%env PYTHONPATH=

env: PYTHONPATH=


In [ ]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6.7-hc745277_3 ...
installing: xz-5.2.4-h14c3975_4 ...
installing: yaml-0.1.7-had09818_2 ...
installing: zlib-1.2.11-ha838bed_2 ...
installing: libedit-3.1.20170329-h6b74fdf_2 ...
installing: readline-7.0-ha6073c6_4 ...
installing: sqlite-3.23.1-he433501_0 ...
installing: asn1crypto-0.24.0-py36_0 ...
installing: certifi-2018.4.16-py36_0 ...
installing: chardet-3.0.4-py36h0f667ec_1 ...
installing: idna-2.6-py36h82fb2a8_1 ...
installing: pycosat-0.6.3-py36h0a5515d_0 ...
installing: pycparser-2.18-py36hf9f622e_1 ...
installing: pysocks-1.6.8-py36_0 ...
installing: ruamel_yaml-0.15.37-py36h14c3975_2 ...
installing: six-1.11

--2020-07-29 11:15:40--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh [following]
--2020-07-29 11:15:40--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

     0K .......... .......... .......... .......... ..........  0% 19.7M 3s
    50K .......... .......... .......... .......... ..........  0%

In [ ]:
%%bash
conda install --channel defaults conda python=3.6 --yes
conda update --channel defaults --all --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs: 
    - conda
    - python=3.6


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ncurses-6.2                |       he6710b0_1         1.1 MB
    ca-certificates-2020.6.24  |                0         133 KB
    tqdm-4.47.0                |             py_0          62 KB
    readline-8.0               |       h7b6447c_0         428 KB
    _libgcc_mutex-0.1          |             main           3 KB
    zlib-1.2.11                |       h7b6447c_3         120 KB
    ld_impl_linux-64-2.33.1    |       h53a641e_7         645 KB
    openssl-1.1.1g             |       h7b6447c_0         3.8 MB
    libstdcxx-ng-9.1.0         |       hdf63c60_0         4.0 MB
    urllib3-1.25.9             |             py_0          98 KB
    pip-20.1.1                 |           py36_1   

ncurses-6.2          |  1.1 MB | ########## | 100% 
ca-certificates-2020 |  133 KB | ########## | 100% 
tqdm-4.47.0          |   62 KB | ########## | 100% 
readline-8.0         |  428 KB | ########## | 100% 
_libgcc_mutex-0.1    |    3 KB | ########## | 100% 
zlib-1.2.11          |  120 KB | ########## | 100% 
ld_impl_linux-64-2.3 |  645 KB | ########## | 100% 
openssl-1.1.1g       |  3.8 MB | ########## | 100% 
libstdcxx-ng-9.1.0   |  4.0 MB | ########## | 100% 
urllib3-1.25.9       |   98 KB | ########## | 100% 
pip-20.1.1           |  2.0 MB | ########## | 100% 
wheel-0.34.2         |   49 KB | ########## | 100% 
chardet-3.0.4        |  197 KB | ########## | 100% 
libffi-3.3           |   54 KB | ########## | 100% 
tk-8.6.10            |  3.2 MB | ########## | 100% 
yaml-0.2.5           |   87 KB | ########## | 100% 
ruamel_yaml-0.15.87  |  256 KB | ########## | 100% 
six-1.15.0           |   13 KB | ########## | 100% 
sqlite-3.32.3        |  2.0 MB | ########## | 100% 
pycparser-2.

In [ ]:
import sys

_ = (sys.path
        .append("/usr/local/lib/python3.6/site-packages"))

In [ ]:
!conda install --channel conda-forge featuretools --yes

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - featuretools


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bokeh-2.1.1                |   py36h9f0ad1d_0         6.9 MB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    click-7.1.2                |     pyh9f0ad1d_0          64 KB  conda-forge
    cloudpickle-1.5.0          |             py_0          22 KB  conda-forge
    conda-4.8.3                |   py36h9f0ad1d_1         3.0 MB  conda-forge
    contextvars-2.4            |             py_0          11 KB  conda-forge
    cytoolz-0.10.1             |   py36h516909a_0         431 KB  conda-forge
   

In [ ]:
!conda install -c conda-forge tensorflow

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - tensorflow


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    absl-py-0.9.0              |           py36_0         162 KB  conda-forge
    astor-0.8.1                |     pyh9f0ad1d_0          25 KB  conda-forge
    c-ares-1.16.1              |       h516909a_0         108 KB  conda-forge
    gast-0.3.3                 |             py_0          12 KB  conda-forge
    google-pasta-0.2.0         |     pyh8c360ce_0          42 KB  conda-forge
    grpcio-1.30.0              |   py36h769ab6c_0         2.0 MB  conda-forge
    h5py-2.10.0                |nompi_py36hecadee3_104         1.1 MB  conda-forge
    hdf5-1.10.6                |nompi_h3c11f04_100         3.0 MB 

In [ ]:
!pip3 install sentencepiece
!pip3 install tf-sentencepiece
!wget -c https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
!chmod +x Anaconda3-5.1.0-Linux-x86_64.sh
!bash ./Anaconda3-5.1.0-Linux-x86_64.sh -b -f -p /usr/local
!conda install -q -y --prefix /usr/local -c pytorch -c tensorcomp tensor_comprehensions

import sys
sys.path.append('/usr/local/lib/python3.6/site-packages/')


/bin/bash: pip3: command not found
/bin/bash: pip3: command not found
--2020-07-29 11:37:23--  https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/archive/Anaconda3-5.1.0-Linux-x86_64.sh [following]
--2020-07-29 11:37:23--  https://repo.anaconda.com/archive/Anaconda3-5.1.0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

PREFIX=/usr/local
installing: python-3.6.4-hc3d631a_1 ...
Python 3.6.4 :: Anaconda,

In [ ]:
!pip install tensorflow_text

    100% |████████████████████████████████| 71kB 4.3MB/s 
    100% |████████████████████████████████| 13.4MB 96kB/s 
    100% |████████████████████████████████| 1.3MB 1.0MB/s 
    100% |████████████████████████████████| 71kB 11.9MB/s 
    100% |████████████████████████████████| 92kB 10.4MB/s 
    100% |████████████████████████████████| 51kB 10.3MB/s 
    100% |████████████████████████████████| 163kB 7.3MB/s 
    100% |████████████████████████████████| 153kB 7.5MB/s 
    100% |████████████████████████████████| 81kB 9.7MB/s 
  Running setup.py bdist_wheel for wrapt ... done
  Stored in directory: /root/.cache/pip/wheels/b1/c2/ed/d62208260edbd3fa7156545c00ef966f45f2063d0a84f8208a
Successfully built wrapt
  Found existing installation: numpy 1.14.2
    Uninstalling numpy-1.14.2:
      Successfully uninstalled numpy-1.14.2
  Found existing installation: wrapt 1.10.11
    DEPRECATION: Uninstalling a distutils installed project (wrapt) has been deprecated and will be removed in a future versi

In [ ]:
!pip install tensorflow_text>=2.0.0rc0
!pip install sentencepiece tf-sentencepiece
import tensorflow as tf
import tensorflow_text
use = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

In [ ]:
X_train = []
for r in tqdm(train.text.values):
  emb = use(r)
  review_emb = tf.reshape(emb, [-1]).numpy()
  X_train.append(review_emb)

X_train = np.array(X_train)
y_train = train.target.values

X_test = []
for r in tqdm(test.text.values):
  emb = use(r)
  review_emb = tf.reshape(emb, [-1]).numpy()
  X_test.append(review_emb)

X_test = np.array(X_test)

100%|██████████| 3263/3263 [03:08<00:00, 17.34it/s]


Training and Evaluating

In [ ]:
train_arrays, test_arrays, train_labels, test_labels = train_test_split(X_train,
                                                                        y_train,
                                                                        random_state =42,
                                                                        test_size=0.20)

In [ ]:
def svc_param_selection(X, y, nfolds):

#    Cs = [1.35, 1.40, 1.45]
#    gammas = [2.15, 2.20, 2.25, 2.30]    best params: {'C': 1.4, 'gamma': 2.25}
    
    Cs = [1.40]
    gammas = [2.25] 

    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=nfolds, n_jobs=8)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search

model = svc_param_selection(train_arrays,train_labels, 10)

In [ ]:
model.best_params_

{'C': 1.4, 'gamma': 2.25}

In [ ]:
pred = model.predict(test_arrays)

In [ ]:
cm = confusion_matrix(test_labels,pred)
cm

array([[808,  66],
       [152, 497]])

In [ ]:
accuracy = accuracy_score(test_labels,pred)
accuracy

0.8568614576493763

SVM prediction

In [ ]:
test_pred = model.predict(X_test)
submission['target'] = test_pred.round().astype(int)

Using keywords for better prediction

In [ ]:
train_df_copy = train
train_df_copy = train_df_copy.fillna('None')
ag = train_df_copy.groupby('keyword').agg({'text':np.size, 'target':np.mean}).rename(columns={'text':'Count', 'target':'Disaster Probability'})

ag.sort_values('Disaster Probability', ascending=False).head(20)

,Count,Disaster Probability
keyword,,
derailment,39,1.000000
debris,37,1.000000
wreckage,39,1.000000
outbreak,40,0.975000
oil%20spill,38,0.973684
typhoon,38,0.973684
suicide%20bombing,33,0.969697
suicide%20bomber,31,0.967742
bombing,29,0.931034


In [ ]:
count = 2
prob_disaster = 0.9
keyword_list_disaster = list(ag[(ag['Count']>count) & (ag['Disaster Probability']>=prob_disaster)].index)
#we print the list of keywords which will be used for prediction correction 
keyword_list_disaster

['bombing',
 'debris',
 'derailment',
 'evacuated',
 'nuclear%20disaster',
 'oil%20spill',
 'outbreak',
 'rescuers',
 'suicide%20bomb',
 'suicide%20bomber',
 'suicide%20bombing',
 'typhoon',
 'wreckage']

In [ ]:
ids_disaster = test['id'][test.keyword.isin(keyword_list_disaster)].values
submission['target'][submission['id'].isin(ids_disaster)] = 1

In [ ]:

submission.to_csv("submission.csv", index=False)
submission.head(10)

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
5,12,1
6,21,0
7,22,0
8,27,0
9,29,0


/content/submission.csv the resultant csv file